In [1]:
import cv2
import plotly.express as px
import numpy as np
import pandas as pd

def read_frames(path: str, max_frames: int = 5000, resize=(100,100)):
    stream = cv2.VideoCapture(path)
    frames = [cv2.resize(stream.read()[1].astype(float)/255, resize) for i in range(max_frames)]
    return np.stack(frames)



In [2]:
def get_key_frames(frames, threshold=0.2):
    diffs = frames[1:, :, :, :] - frames[:-1, :, :, :]
    delta = np.abs(diffs).mean(axis=(1,2,3))
    frames_indexes = np.where(delta>threshold)[0] + 1
    return frames[frames_indexes, :, :, :], frames_indexes

In [3]:
import hashlib

def hash_image(image, res=8):
    image = image.mean(axis=-1)
    image = cv2.resize(image, dsize=(res, res), interpolation=cv2.INTER_AREA)
    image = image > image.mean()
    return hashlib.md5(image).hexdigest()
    

In [4]:
paths = ['../static/The Office/season 4/The Office (US) (2005) - S04E01-E02 - Fun Run (1080p BluRay x265 Silence).mkv',
         '../static/The Office/season 4/The Office (US) (2005) - S04E03-E04 - Dunder Mifflin Infinity (1080p BluRay x265 Silence).mkv']
frames = [read_frames(path) for path in paths]

In [11]:
class IntroDetector:
    def __init__(self, hash_options: dict=None):
        self.frame_hashes = {}
        if hash_options is None:
            hash_options = {}
        self.hash_options = hash_options
        self.keyframes = {}
        
    def update(self, frames, name: str):
        keyframes, frame_indexes = get_key_frames(frames)
        self.keyframes[name] = []
        for kf_index, (kf, ts) in enumerate(zip(keyframes, frame_indexes)):
            h = hash_image(kf, **self.hash_options)
            if h not in self.frame_hashes:
                self.frame_hashes[h] = []
            self.frame_hashes[h].append((name, kf_index))
            self.keyframes[name].append({'ts': ts, 'kf': kf})
            
    def detect(self, threshold=0.2, morph=2):
        shot_repeats_for_intro = max(2, int(len(self.keyframes) * threshold))
        for frame_hash, occurrences in self.frame_hashes.items():
            for name, kf_index in occurrences:
                self.keyframes[name][kf_index]['count'] = len(occurrences)
        result = {}
        for name, keyframes in self.keyframes.items():
            mask = np.array([kf_dict['count']>=shot_repeats_for_intro for kf_dict in keyframes])
            if morph > 0:
                mask = erode(dilate(mask, morph), morph)
            
            data = self.keyframes[name]
            for d, m in zip(data, mask):
                d['intro'] = m
            # TODO: dont use pandas, for some reason slow af
            result[name] = pd.DataFrame(data).drop(columns=['kf'])
            
        
        return result
            
            

In [12]:
idet = IntroDetector()
idet.update(frames[0], '1-2')
idet.update(frames[1], '3-4')

In [ ]:
res = idet.detect()

In [8]:
def dilate(a: np.array, size: int = 1):
    kernel_size = 2 * size + 1
    ii = np.arange(a.shape[0]) + (np.arange(kernel_size) - size)[:, None]
    ii[ii<0] = 0
    ii[ii>=a.shape[0]] = a.shape[0] - 1
    return a[ii].any(axis=0)

def erode(a: np.array, size: int = 1):
    kernel_size = 2 * size + 1
    ii = np.arange(a.shape[0]) + (np.arange(kernel_size) - size)[:, None]
    ii[ii<0] = 0
    ii[ii>=a.shape[0]] = a.shape[0] - 1
    return a[ii].all(axis=0)

In [10]:
res['1-2'].columns

Index(['ts', 'kf', 'count', 'intro'], dtype='object')